In [1]:
from text2brick.dataset import LegoDatasetGenerator
from text2brick.gym import IoUValidityRewardFunc
reward_function = IoUValidityRewardFunc()
generator = LegoDatasetGenerator(reward_function=reward_function)

In [32]:
generator.generate_dataset(50)

100%|██████████| 50/50 [00:30<00:00,  1.63it/s]


In [33]:
from text2brick.dataset import LegoPretrainDataset
from torch_geometric.loader import DataLoader 
import torch

BATCH_SIZE = 10

dataset = LegoPretrainDataset("./lego_dataset")
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [34]:
c = 0
for i, batch in enumerate(dataloader):
    #print(batch[4].batch)

    if batch[4].batch.max() < BATCH_SIZE - 1:
        c += 1
        print("Error count: ", c)
        print(i)
        print(batch[4].batch)
        print(batch[4].edge_index)
        print(batch[4].x)
print(i)

Error count:  1
6
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5,
        5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8,

In [20]:
print(dataset.__getitem__(12))

(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -

In [35]:
from text2brick.gym import Text2Brick_v1
model = Text2Brick_v1()

Using cache found in C:\Users\ebern/.cache\torch\hub\pytorch_vision_v0.10.0


In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

Using device: cpu


Text2Brick_v1(
  (snn): SNN(
    (cnn): CNN(
      (model): Sequential(
        (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
        (3): Fire(
          (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
          (squeeze_activation): ReLU(inplace=True)
          (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
          (expand1x1_activation): ReLU(inplace=True)
          (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (expand3x3_activation): ReLU(inplace=True)
        )
        (4): Fire(
          (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
          (squeeze_activation): ReLU(inplace=True)
          (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
          (expand1x1_activation): ReLU(inplace=True)
          (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), st

In [37]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define the loss function (CrossEntropyLoss for classification tasks)
x_loss_fn = nn.CrossEntropyLoss()
y_loss_fn = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train_model(model, dataloader, optimizer, x_loss_fn, y_loss_fn, num_epochs=10):
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        
        # Loop over the batches of data
        for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            optimizer.zero_grad()  # Zero the parameter gradients

            image_target = batch[0].to(device)
            current_build_img = batch[1].to(device)
            next_node = batch[2].to(device)
            reward = batch[3].to(device)
            current_graph = batch[4].to(device)
            
            x_pred, y_pred = model(current_build_img, current_graph, reward, image_target=image_target, return_logits=True)
            x_true, y_true = next_node[:, 0], next_node[:, 1]

            # Compute loss for both x and y coordinates
            x_loss = x_loss_fn(x_pred, x_true)
            y_loss = y_loss_fn(y_pred, y_true)

            # Combine the losses (you can weight them differently if needed)
            loss = x_loss + y_loss
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # Print epoch loss
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Run the training loop
train_model(model, dataloader, optimizer, x_loss_fn, y_loss_fn, num_epochs=10)


Epoch 1/10:   0%|          | 1/263 [00:01<05:01,  1.15s/it]

Shape of f_fused: torch.Size([10, 13, 13])
Shape of h_graph: torch.Size([10, 64])
Shape of reward: torch.Size([10])
Shape of f_fused_flat: torch.Size([10, 169])
Shape of h_graph_flat: torch.Size([10, 64])
Shape of reward_flat: torch.Size([10, 1])


Epoch 1/10:   1%|          | 2/263 [00:02<04:25,  1.02s/it]

Shape of f_fused: torch.Size([10, 13, 13])
Shape of h_graph: torch.Size([10, 64])
Shape of reward: torch.Size([10])
Shape of f_fused_flat: torch.Size([10, 169])
Shape of h_graph_flat: torch.Size([10, 64])
Shape of reward_flat: torch.Size([10, 1])


Epoch 1/10:   1%|          | 3/263 [00:02<04:06,  1.05it/s]

...

Epoch 1/10:  35%|███▌      | 93/263 [01:25<03:08,  1.11s/it]

Shape of f_fused: torch.Size([10, 13, 13])
Shape of h_graph: torch.Size([10, 64])
Shape of reward: torch.Size([10])
Shape of f_fused_flat: torch.Size([10, 169])
Shape of h_graph_flat: torch.Size([10, 64])
Shape of reward_flat: torch.Size([10, 1])


Epoch 1/10:  35%|███▌      | 93/263 [01:26<02:38,  1.07it/s]

Shape of f_fused: torch.Size([10, 13, 13])
Shape of h_graph: torch.Size([9, 64])
Shape of reward: torch.Size([10])
Shape of f_fused_flat: torch.Size([10, 169])
Shape of h_graph_flat: torch.Size([9, 64])
Shape of reward_flat: torch.Size([10, 1])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 10 but got size 9 for tensor number 1 in the list.